In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
# ライブラリのインポート
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# データの読み込み
train_df = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')

# 欠損値の確認
print(train_df.isnull().sum())

# 数値データとカテゴリカルデータの分離
numeric_features = train_df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train_df.select_dtypes(include=['object']).columns

# 'Price' 列を numeric_features から除外
numeric_features = numeric_features.drop('Price')

# 数値データの欠損値処理
imputer = SimpleImputer(strategy='mean')
train_df[numeric_features] = imputer.fit_transform(train_df[numeric_features])
test_df[numeric_features] = imputer.transform(test_df[numeric_features])

# カテゴリカルデータのワンホットエンコーディング
train_df = pd.get_dummies(train_df, columns=categorical_features)
test_df = pd.get_dummies(test_df, columns=categorical_features)

# train_df と test_df の列を揃える
train_df, test_df = train_df.align(test_df, join='left', axis=1, fill_value=0)

# 特徴量とターゲットの分離
X = train_df.drop('Price', axis=1)
y = train_df['Price']

# 訓練データとテストデータの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# モデルの構築
model = RandomForestRegressor(n_estimators=100, random_state=42)

# モデルのトレーニング
model.fit(X_train, y_train)

# 予測の実行
y_pred = model.predict(X_val)

# 評価指標の計算
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {mse}')

# テストデータに対する予測
test_pred = model.predict(test_df.drop('Price', axis=1, errors='ignore'))

# 結果の保存
submission = pd.DataFrame({'id': test_df['id'], 'Price': test_pred})
submission.to_csv('submission.csv', index=False)

# 出力ファイルの最初と最後の6行を表示
print("First 6 rows of the submission file:")
print(submission.head(6))

print("Last 6 rows of the submission file:")
print(submission.tail(6))



id                         0
Brand                   9705
Material                8347
Size                    6595
Compartments               0
Laptop Compartment      7444
Waterproof              7050
Style                   7970
Color                   9950
Weight Capacity (kg)     138
Price                      0
dtype: int64
Mean Squared Error: 1562.7838038845634
First 6 rows of the submission file:
         id      Price
0  300000.0  87.345029
1  300001.0  87.440523
2  300002.0  97.510579
3  300003.0  83.585873
4  300004.0  76.720012
5  300005.0  87.142592
Last 6 rows of the submission file:
              id      Price
199994  499994.0  75.330072
199995  499995.0  89.036235
199996  499996.0  82.112442
199997  499997.0  83.490923
199998  499998.0  86.697409
199999  499999.0  73.119162
